In this project you will apply what you've learned about robot dynamics to simulate the motion of a robot arm.  You will write code that builds on the Modern Robotics library to simulate the motion of the robot, and you will visualize the resulting motion of the robot by making videos using CoppeliaSim.

The UR5 is a popular 6-DOF industrial robot arm.  The robot has geared motors at each joint, but in this project, we ignore the effects of gearing, such as friction and the increased apparent inertia of the rotor.  


The relevant kinematic and inertial parameters of the UR5 are:

In [1]:
import math
import numpy as np
import modern_robotics as mr
import csv

pi=math.pi

M01=np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0.089159],
              [0, 0, 0, 1]])
M12=np.array([[0,0,1,0.28],
              [0,1,0,0.13585],
              [-1,0,0,0],
              [0,0,0,1]])
M23=np.array([[1,0,0,0],
              [0,1,0,-0.1197],
              [0,0,1,0.395],
              [0,0,0,1]])
M34=np.array([[0,0,1,0],
              [0,1,0,0],
              [-1,0,0,0.14225],
              [0,0,0,1]])
M45=np.array([[1,0,0,0],
              [0,1,0,0.093],
              [0,0,1,0],
              [0,0,0,1]])
M56=np.array([[1,0,0,0],
              [0,1,0,0],
              [0,0,1,0.09465],
              [0,0,0,1]])
M67=np.array([[1,0,0,0],
              [0,0,1,0.0823],
              [0,-1,0,0],
              [0,0,0,1]])
print('Link frame 0: \n', M01)
print('\nLink frame 1: \n',M12)
print('\nLink frame 2: \n',M23)
print('\nLink frame 3: \n',M34)
print('\nLink frame 4: \n',M45)
print('\nLink frame 5: \n',M56)
print('\nLink frame 6: \n',M67)

Link frame 0: 
 [[1.       0.       0.       0.      ]
 [0.       1.       0.       0.      ]
 [0.       0.       1.       0.089159]
 [0.       0.       0.       1.      ]]

Link frame 1: 
 [[ 0.       0.       1.       0.28   ]
 [ 0.       1.       0.       0.13585]
 [-1.       0.       0.       0.     ]
 [ 0.       0.       0.       1.     ]]

Link frame 2: 
 [[ 1.      0.      0.      0.    ]
 [ 0.      1.      0.     -0.1197]
 [ 0.      0.      1.      0.395 ]
 [ 0.      0.      0.      1.    ]]

Link frame 3: 
 [[ 0.       0.       1.       0.     ]
 [ 0.       1.       0.       0.     ]
 [-1.       0.       0.       0.14225]
 [ 0.       0.       0.       1.     ]]

Link frame 4: 
 [[1.    0.    0.    0.   ]
 [0.    1.    0.    0.093]
 [0.    0.    1.    0.   ]
 [0.    0.    0.    1.   ]]

Link frame 5: 
 [[1.      0.      0.      0.     ]
 [0.      1.      0.      0.     ]
 [0.      0.      1.      0.09465]
 [0.      0.      0.      1.     ]]

Link frame 6: 
 [[ 1.      0.      0

In [2]:
Mlist=np.array([M01, M12, M23, M34, M45, M56, M67])
G1=np.diag([0.01026749,0.01026749,0.00666,3.7,3.7,3.7])
G2=np.diag([0.22689,0.22689,0.0151074,8.393,8.393,8.393])
G3=np.diag([0.0494433,0.0494433,0.004095,2.275,2.275,2.275])
G4=np.diag([0.1111727,0.1111727,0.21942,1.219,1.219,1.219])
G5=np.diag([0.1111727,0.1111727,0.21942,1.219,1.219,1.219])
G6=np.diag([0.017136,0.017136,0.033822,0.1879,0.1879,0.1879])
print('Spatial Inertia Matrix of Link 1: \n', G1)
print('\nSpatial Inertia Matrix of Link 2: \n', G2)
print('\nSpatial Inertia Matrix of Link 3: \n', G3)
print('\nSpatial Inertia Matrix of Link 4: \n', G4)
print('\nSpatial Inertia Matrix of Link 5: \n', G5)
print('\nSpatial Inertia Matrix of Link 6: \n', G6)

Spatial Inertia Matrix of Link 1: 
 [[0.01026749 0.         0.         0.         0.         0.        ]
 [0.         0.01026749 0.         0.         0.         0.        ]
 [0.         0.         0.00666    0.         0.         0.        ]
 [0.         0.         0.         3.7        0.         0.        ]
 [0.         0.         0.         0.         3.7        0.        ]
 [0.         0.         0.         0.         0.         3.7       ]]

Spatial Inertia Matrix of Link 2: 
 [[0.22689   0.        0.        0.        0.        0.       ]
 [0.        0.22689   0.        0.        0.        0.       ]
 [0.        0.        0.0151074 0.        0.        0.       ]
 [0.        0.        0.        8.393     0.        0.       ]
 [0.        0.        0.        0.        8.393     0.       ]
 [0.        0.        0.        0.        0.        8.393    ]]

Spatial Inertia Matrix of Link 3: 
 [[0.0494433 0.        0.        0.        0.        0.       ]
 [0.        0.0494433 0.        0

In [3]:
Glist = np.array([G1, G2, G3, G4, G5, G6])
SList=np.array([[0,0,0,0,0,0],
                [0,1,1,1,0,1],
                [1,0,0,0,-1,0],
                [0,-0.089159,-0.089159,-0.089159,-0.10915,0.005491],
                [0,0,0,0,0.81725,0],
                [0,0,0.425,0.81725,0,0.81725]])
print('Screw axes of the Joints: \n',SList)

Screw axes of the Joints: 
 [[ 0.        0.        0.        0.        0.        0.      ]
 [ 0.        1.        1.        1.        0.        1.      ]
 [ 1.        0.        0.        0.       -1.        0.      ]
 [ 0.       -0.089159 -0.089159 -0.089159 -0.10915   0.005491]
 [ 0.        0.        0.        0.        0.81725   0.      ]
 [ 0.        0.        0.425     0.81725   0.        0.81725 ]]


Your job is to write code that simulates the motion of the UR5 for a specified amount of time (in seconds), from a specified initial configuration (at zero velocity), when zero torques are applied to the joints.  In other words, the robot simply falls in gravity. Gravity is g = 9.81g=9.81m/s^2 in the z-direction, i.e., gravity acts downward.   The motion should be simulated with at least 100 integration steps per second.  Your program should calculate and record the robot joint angles at each step. This data should be saved as a .csv file, where each row has six numbers separated by commas. This .csv file is suitable for animation with the CoppeliaSim UR5 csv animation scene.

You will perform two simulations and make videos of each:


1. The robot falling from the zero (home) configuration for 3 seconds.
2. The robot falling from a configuration where all joints are at their zero position, except for joint 2, which is at −1 radian.  This simulation should last 5 seconds.


Important:  Since the simulated robot has no friction and zero motor torques, no energy is added or subtracted during the simulated motion.  Therefore, the total energy of the robot (kinetic plus potential) must remain constant during the simulation.  If you see the robot swinging higher and higher, or losing energy, something is wrong wih your simulation.

In [4]:
g=np.array([0,0,-9.81])
thetalist=np.array([0,0,0,0,0,0])
dthetalist=np.array([0,0,0,0,0,0])
ddthetalist=np.array([0,0,0,0,0,0])
Tau=np.array([0,0,0,0,0,0])
Ftip=np.array([0,0,0,0,0,0])

In [5]:
def iterations(thetalist, dthetalist, tau, g, Ftip, Mlist, Glist, Slist, dt):
    ddthetalist=mr.ForwardDynamics(thetalist, dthetalist, tau, g, Ftip, Mlist, Glist, Slist)
    new_theta, new_dtheta=mr.EulerStep(thetalist, dthetalist,ddthetalist, dt)
    return new_theta, new_dtheta

In [6]:
#Simulation 1
t=2
steps=400
dt=t/steps
outfile=open('Simulation_1.csv','w')
write=csv.writer(outfile)
write.writerow(thetalist)
for s in range(steps):
    new_thetalist, new_dthetalist=iterations(thetalist,dthetalist,Tau,g,Ftip,Mlist,Glist,SList,dt)
    write.writerow(new_thetalist)
    thetalist=new_thetalist
    dthetalist=new_dthetalist

In [7]:
#Simulation 2
g=np.array([0,0,-9.81])
thetalist=np.array([0,-1,0,0,0,0])
dthetalist=np.array([0,0,0,0,0,0])
ddthetalist=np.array([0,0,0,0,0,0])
Tau=np.array([0,0,0,0,0,0])
Ftip=np.array([0,0,0,0,0,0])
t=5
steps=1000
dt=t/steps
outfile=open('Simulation_2.csv','w')
write=csv.writer(outfile)
write.writerow(thetalist)
for s in range(steps):
    new_thetalist,new_dthetalist=iterations(thetalist,dthetalist,Tau,g,Ftip,Mlist,Glist,SList,dt)
    write.writerow(new_thetalist)
    thetalist=new_thetalist
    dthetalist=new_dthetalist